In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the news website
url = "https://www.cnbc.com/world/?region=world"

# Send a GET request to fetch the webpage content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract all paragraphs from the webpage
paragraphs = soup.find_all('p')
text_content = ' '.join([para.get_text() for para in paragraphs])

# Print the extracted text content
print(text_content[:1000])  # Preview first 1000 characters

Credit Cards Loans Banking Mortgages Insurance Credit Monitoring Personal Finance Small Business Taxes Help for Low Credit Scores Investing SELECT All Credit Cards Find the Credit Card for You Best Credit Cards Best Rewards Credit Cards Best Travel Credit Cards Best 0% APR Credit Cards Best Balance Transfer Credit Cards Best Cash Back Credit Cards Best Credit Card Welcome Bonuses Best Credit Cards to Build Credit SELECT All Loans Find the Best Personal Loan for You Best Personal Loans Best Debt Consolidation Loans Best Loans to Refinance Credit Card Debt Best Loans with Fast Funding Best Small Personal Loans Best Large Personal Loans Best Personal Loans to Apply Online Best Student Loan Refinance SELECT All Banking Find the Savings Account for You Best High Yield Savings Accounts Best Big Bank Savings Accounts Best Big Bank Checking Accounts Best No Fee Checking Accounts No Overdraft Fee Checking Accounts Best Checking Account Bonuses Best Money Market Accounts Best CDs Best Credit Uni

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK tokenizers
nltk.download('punkt')

# Tokenize the text content
tokens = word_tokenize(text_content)

# Print some tokens for preview
print(tokens[:20])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['Credit', 'Cards', 'Loans', 'Banking', 'Mortgages', 'Insurance', 'Credit', 'Monitoring', 'Personal', 'Finance', 'Small', 'Business', 'Taxes', 'Help', 'for', 'Low', 'Credit', 'Scores', 'Investing', 'SELECT']


In [ ]:
import nltk
from nltk.corpus import conll2002
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split

# Download necessary NLTK data
nltk.download('conll2002')

# Load the dataset (Spanish version in this example)
train_sents = list(conll2002.iob_sents('esp.train'))
test_sents = list(conll2002.iob_sents('esp.testb'))

# Prepare sentences and labels
def prepare_data(sents):
    sentences = []
    labels = []
    for sent in sents:
        words = [word for word, pos, tag in sent]
        tags = [tag for word, pos, tag in sent]
        sentences.append(words)
        labels.append(tags)
    return sentences, labels

train_sentences, train_labels = prepare_data(train_sents)
test_sentences, test_labels = prepare_data(test_sents)

# Create a word and tag dictionary
words = list(set(word.lower() for sent in train_sentences for word in sent))
tags = list(set(tag for sent in train_labels for tag in sent))

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1  # Unknown words
word2idx["PAD"] = 0  # Padding

tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: t for t, i in tag2idx.items()}

# Convert words and tags into integer indices
max_len = 75  # Max sentence length
X_train = [[word2idx.get(w.lower(), word2idx["UNK"]) for w in s] for s in train_sentences]
X_test = [[word2idx.get(w.lower(), word2idx["UNK"]) for w in s] for s in test_sentences]

# Pad sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

# Convert labels to categorical data and pad
y_train = [[tag2idx[tag] for tag in label] for label in train_labels]
y_test = [[tag2idx[tag] for tag in label] for label in test_labels]

y_train = pad_sequences(y_train, maxlen=max_len, padding='post', value=tag2idx["O"])
y_test = pad_sequences(y_test, maxlen=max_len, padding='post', value=tag2idx["O"])

# Convert labels to one-hot encoding
y_train = [to_categorical(i, num_classes=len(tags)) for i in y_train]
y_test = [to_categorical(i, num_classes=len(tags)) for i in y_test]

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print(f"Number of sentences in training: {len(X_train)}")
print(f"Number of sentences in validation: {len(X_val)}")

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


Number of sentences in training: 7490
Number of sentences in validation: 833


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional, Dropout

# Ensure TensorFlow is in eager execution mode
tf.config.run_functions_eagerly(True)

# Define model parameters
input_dim = len(word2idx)  # Vocabulary size
output_dim = 64  # Embedding output size
n_tags = len(tag2idx)  # Number of unique tags
max_len = 75  # Max sentence length

# Build the model
model = Sequential()

# Embedding layer (no input_length)
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, mask_zero=True))

# Bidirectional LSTM layer
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))

# TimeDistributed layer to output for each token (ensure compatibility)
model.add(TimeDistributed(Dense(n_tags, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=3, validation_data=(X_val, np.array(y_val)))

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/3
235/235 ━━━━━━━━━━━━━━━━━━━━ 562s 2s/step - accuracy: 0.9394 - loss: 0.8727 - val_accuracy: 0.9604 - val_loss: 0.3438
Epoch 2/3
235/235 ━━━━━━━━━━━━━━━━━━━━ 630s 2s/step - accuracy: 0.9623 - loss: 0.3024 - val_accuracy: 0.9705 - val_loss: 0.2444
Epoch 3/3
235/235 ━━━━━━━━━━━━━━━━━━━━ 564s 2s/step - accuracy: 0.9736 - loss: 0.2063 - val_accuracy: 0.9749 - val_loss: 0.2059


In [ ]:
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, np.array(y_test))
print(f"Test Accuracy: {test_accuracy}")

# Predict the Named Entities in a test sentence
i = 0  # Choose a sentence index
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)

print("Test Sentence:", ' '.join([list(word2idx.keys())[list(word2idx.values()).index(idx)] for idx in X_test[i]]))
print("Predicted entities:", [idx2tag[idx] for idx in p[0]])

48/48 ━━━━━━━━━━━━━━━━━━━━ 46s 948ms/step - accuracy: 0.9705 - loss: 0.2326
Test Accuracy: 0.9710129499435425
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 734ms/step
Test Sentence: la coruña , 23 may ( efecom ) . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
Predicted entities: ['O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### **Model Training Output:**


- **48/48**: This means that 48 batches of data were processed during training.
- **59s 1s/step**: The entire epoch took 59 seconds, and each batch took about 1 second to process.
- **accuracy: 0.9709**: The model achieved an accuracy of 97.09% during training.
- **loss: 0.2333**: The loss value is 0.2333, which indicates how well the model fits the data. Lower loss typically means better performance.


- **Test Accuracy**: The accuracy on the test set is 97.11%, which suggests that the model is performing very well on unseen data.

### **Test Sentence and Predicted Entities:**



- **Test Sentence**: The model was tested on the sentence: `"la coruña , 23 may ( efecom ) ."`. The `PAD` tokens are padding to ensure a fixed length sequence for the model, and they are ignored.
- **Predicted Entities**:
  - `O`: "Outside" – no named entity.
  - `B-LOC`: "Beginning of a location entity" – `"la coruña"` is identified as a location (the "B-LOC" tag indicates the beginning of this entity).
  - `B-ORG`: "Beginning of an organization entity" – `"efecom"` is identified as an organization.

The model successfully identified `"la coruña"` as a location (`B-LOC`) and `"efecom"` as an organization (`B-ORG`). All other tokens in the sentence were marked as outside any named entity (`O`).

### Overall Interpretation:
The model has learned to recognize named entities in text well, as reflected by the high accuracy (97.1%) both during training and testing. The predictions for the test sentence indicate that the model correctly identified entities like locations (LOC) and organizations (ORG).**